In [2]:
import torch
torch.cuda.is_available()



False

In [1]:
#load models
from FlagEmbedding import FlagReranker
from sentence_transformers import CrossEncoder

# Initialize the reranker
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True)  # Setting use_fp16 to True speeds up computation with a slight performance degradation

# Now standard SBERT
# Initialize the cross-encoder model
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)


/workspaces/KernelMemory.Extensions/src/python/KernelMemory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Define the question and possible answers
question = "Please explain cross encoder in the context of artificial intelligence"
sentences = [
    "You can find lots of transformers in the huggingface site, as well as cross encoders",
    "Maples trees lives a really long number of years, but you need to take care of them, place a cross on the tree.",
    "i'm an enthusiast of AI especially of generative AI because it gives a lots of new techniques for Natural Language Processing",
    "huggingface is the facto the place of custom models.",
    "please explain cross encoder",
    "A cross encoder is a type of neural network architecture used in natural language processing tasks. It takes pairs of sentences as input and produces a single fixed-length vector representation that captures the semantic similarity between the two sentences.",
    "In the context of crossing roads, a cross encoder refers to a device or system that controls the timing of traffic lights at intersections, ensuring the safe crossing of vehicles and pedestrians",
    "Cross encoder is a term used in transportation engineering to describe a technique for optimizing the design of road intersections, taking into account factors such as traffic flow, safety, and efficiency of movement across different roads."
]


In [4]:

# Compute scores for the question and possible answers
scores = reranker.compute_score([[question, answer] for answer in sentences])


In [5]:

# Compute scores for the question and possible answers
scoressbert = model.predict([(question, answer) for answer in sentences])


In [7]:
# Now cohere
import cohere
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

co = cohere.Client(os.getenv("COHERE_API_KEY"))

coresults = co.rerank(query=question, documents=sentences, top_n=10, model='rerank-english-v2.0', return_documents=False) # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(coresults.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index} Relevance Score: {r.relevance_score:.2f}")


Document Rank: 1, Document Index: 5 Relevance Score: 1.00
Document Rank: 2, Document Index: 6 Relevance Score: 0.99
Document Rank: 3, Document Index: 7 Relevance Score: 0.95
Document Rank: 4, Document Index: 0 Relevance Score: 0.40
Document Rank: 5, Document Index: 4 Relevance Score: 0.21
Document Rank: 6, Document Index: 2 Relevance Score: 0.02
Document Rank: 7, Document Index: 3 Relevance Score: 0.02
Document Rank: 8, Document Index: 1 Relevance Score: 0.00


In [8]:

print(scores)
print(scoressbert)

# now print all the index sorted in descending order based on the value of scores and scoresbert
print("Scores")
print(sorted(range(len(scores)), key=lambda i: scores[i], reverse=True))
print("ScoresBert")
print(sorted(range(len(scoressbert)), key=lambda i: scoressbert[i], reverse=True))
print("ScoresCohere")
print([r.index for r in coresults.results])


[-5.548015117645264, -9.480554580688477, -8.299059867858887, -9.462210655212402, 9.492735862731934, 0.4065203368663788, -0.42575928568840027, -1.9448825120925903]
[ -3.51413   -11.317091   -9.698744  -11.310938    5.5058546   5.1461453
   3.9989958   3.7147536]
Scores
[4, 5, 6, 7, 0, 2, 3, 1]
ScoresBert
[4, 5, 6, 7, 0, 2, 3, 1]
ScoresCohere
[5, 6, 7, 0, 4, 2, 3, 1]


In [16]:

# Now normalize the scores in range 0 1
import numpy as np
scores = np.array(scores)
scoressbert = np.array(scoressbert)
scores = (scores - scores.min()) / (scores.max() - scores.min())
scoressbert = (scoressbert - scoressbert.min()) / (scoressbert.max() - scoressbert.min())

# print with maximum 3 decimal places
np.set_printoptions(precision=3)

print(scores)
print(scoressbert)


[0.207 0.    0.062 0.001 1.    0.521 0.478 0.397]
[4.638e-01 0.000e+00 9.620e-02 3.659e-04 1.000e+00 9.786e-01 9.104e-01
 8.935e-01]
